# Custom Ramadan time series

In this notebook, we create time series regarding the Ramadan. More precisely, we create time series simply annotating if the days of a certain year are into the Ramadan period or not.

In [1]:
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import glob
plt.style.use("default")

In [2]:
# Add the python path to the folder containing some custom packages.
import sys
sys.path.insert(0, "../../packages/")
from TsIP.TsIP import TsIP

In [3]:
# Define the path where the results arising from this analysis will be saved.
PATH_TO_SAVE_DATA = "./output_timeseries/"

In [4]:
# Read the shapefiles of each country.
path = "../../packages/MapPlot/shapefiles/adminstratas/"
all_folders = glob.glob(path + "*")

dfs = []

for folder in all_folders:
    all_files = glob.glob(folder + "/*.shp")
    for filename in all_files:
        df = gpd.read_file(filename)
        dfs.append(df)

In [5]:
# Concatenate data.
df = pd.concat(dfs, axis = 0, ignore_index = True)
df.head()

,country,region,geometry
0,Burkina Faso,Boucle-Du-Mouhoun,"POLYGON ((-3.18158 13.67744, -3.18109 13.67721..."
1,Burkina Faso,Cascades,"POLYGON ((-5.38923 11.00086, -5.37382 10.97698..."
2,Burkina Faso,Centre,"POLYGON ((-1.60508 12.67250, -1.58739 12.67057..."
3,Burkina Faso,Centre-Est,"POLYGON ((-0.30646 12.55612, -0.30573 12.55588..."
4,Burkina Faso,Centre-Nord,"MULTIPOLYGON (((-0.76975 12.55062, -0.77002 12..."


In [6]:
# Change the name of adminstrata feature.
df.rename(columns = {"region": "AdminStrata", "country": "Country"}, inplace = True)
# Delete geometry feature.
df.drop(columns = "geometry", inplace = True)
# Add a column for the Ramadan feature.
df["Ramadan"] = 0
df.head()

,Country,AdminStrata,Ramadan
0,Burkina Faso,Boucle-Du-Mouhoun,0
1,Burkina Faso,Cascades,0
2,Burkina Faso,Centre,0
3,Burkina Faso,Centre-Est,0
4,Burkina Faso,Centre-Nord,0


## Creation of the time series

In [7]:
# Create multi-index columns dataframe.
df = df.set_index(["Country", "AdminStrata"]).T.reset_index(drop = True)
df.columns = pd.MultiIndex.from_tuples(list(map(lambda x: tuple(list(x) + ["Ramadan"]), df.columns)), 
                                       names = ["Country", "AdminStrata", "Indicator"])
df

Country          Burkina Faso                                          \
AdminStrata Boucle-Du-Mouhoun Cascades  Centre Centre-Est Centre-Nord   
Indicator             Ramadan  Ramadan Ramadan    Ramadan     Ramadan   
0                           0        0       0          0           0   

Country                                                            ...  \
AdminStrata Centre-Ouest Centre-Sud     Est Hauts-Bassins    Nord  ...   
Indicator        Ramadan    Ramadan Ramadan       Ramadan Ramadan  ...   
0                      0          0       0             0       0  ...   

Country       Yemen                                                       \
AdminStrata  Sana'a    Aden    Lahj   Marib Al Mahwit Al Maharah   Amran   
Indicator   Ramadan Ramadan Ramadan Ramadan   Ramadan    Ramadan Ramadan   
0                 0       0       0       0         0          0       0   

Country                                 
AdminStrata Al Dhale'e  Raymah Socotra  
Indicator      Ramadan Ramadan Ramadan  
0                    0       0       0  

[1 rows x 113 columns]

In [8]:
# Define reference dates to build the time series.
FIRST_YEAR = "2018"
LAST_YEAR = "2023"

new_index = pd.date_range(FIRST_YEAR, LAST_YEAR, freq = "D")

In [9]:
# Create static time series of the Ramadan (all 0 values).
df = df.loc[df.index.repeat(len(new_index))]
df.index = new_index
df.index.name = "Datetime"
df

Country          Burkina Faso                                          \
AdminStrata Boucle-Du-Mouhoun Cascades  Centre Centre-Est Centre-Nord   
Indicator             Ramadan  Ramadan Ramadan    Ramadan     Ramadan   
Datetime                                                                
2018-01-01                  0        0       0          0           0   
2018-01-02                  0        0       0          0           0   
2018-01-03                  0        0       0          0           0   
2018-01-04                  0        0       0          0           0   
2018-01-05                  0        0       0          0           0   
...                       ...      ...     ...        ...         ...   
2022-12-28                  0        0       0          0           0   
2022-12-29                  0        0       0          0           0   
2022-12-30                  0        0       0          0           0   
2022-12-31                  0        0       0          0           0   
2023-01-01                  0        0       0          0           0   

Country                                                            ...  \
AdminStrata Centre-Ouest Centre-Sud     Est Hauts-Bassins    Nord  ...   
Indicator        Ramadan    Ramadan Ramadan       Ramadan Ramadan  ...   
Datetime                                                           ...   
2018-01-01             0          0       0             0       0  ...   
2018-01-02             0          0       0             0       0  ...   
2018-01-03             0          0       0             0       0  ...   
2018-01-04             0          0       0             0       0  ...   
2018-01-05             0          0       0             0       0  ...   
...                  ...        ...     ...           ...     ...  ...   
2022-12-28             0          0       0             0       0  ...   
2022-12-29             0          0       0             0       0  ...   
2022-12-30             0          0       0             0       0  ...   
2022-12-31             0          0       0             0       0  ...   
2023-01-01             0          0       0             0       0  ...   

Country       Yemen                                                       \
AdminStrata  Sana'a    Aden    Lahj   Marib Al Mahwit Al Maharah   Amran   
Indicator   Ramadan Ramadan Ramadan Ramadan   Ramadan    Ramadan Ramadan   
Datetime                                                                   
2018-01-01        0       0       0       0         0          0       0   
2018-01-02        0       0       0       0         0          0       0   
2018-01-03        0       0       0       0         0          0       0   
2018-01-04        0       0       0       0         0          0       0   
2018-01-05        0       0       0       0         0          0       0   
...             ...     ...     ...     ...       ...        ...     ...   
2022-12-28        0       0       0       0         0          0       0   
2022-12-29        0       0       0       0         0          0       0   
2022-12-30        0       0       0       0         0          0       0   
2022-12-31        0       0       0       0         0          0       0   
2023-01-01        0       0       0       0         0          0       0   

Country                                 
AdminStrata Al Dhale'e  Raymah Socotra  
Indicator      Ramadan Ramadan Ramadan  
Datetime                                
2018-01-01           0       0       0  
2018-01-02           0       0       0  
2018-01-03           0       0       0  
2018-01-04           0       0       0  
2018-01-05           0       0       0  
...                ...     ...     ...  
2022-12-28           0       0       0  
2022-12-29           0       0       0  
2022-12-30           0       0       0  
2022-12-31           0       0       0  
2023-01-01           0       0       0  

[1827 rows x 113 columns]

In [10]:
# Define a list of Ramadan dates.
RAMADAN_DATES = [("2018-05-16", "2018-06-14"), ("2019-05-5", "2019-06-3"), 
                 ("2020-04-23", "2020-05-23"), ("2021-04-12", "2021-05-12"), 
                 ("2022-04-01", "2022-05-01")]

In [11]:
# Add the value 1 in correspondence of Ramadan dates.
for ramadan_date in RAMADAN_DATES:
    df.loc[ramadan_date[0]:ramadan_date[1]] = 1

In [12]:
# Plot static time series.
TsIP(df).interactive_plot_df(matplotlib = False, style = "lines", yaxis = "Ramadan")

interactive(children=(ToggleButtons(description='Country', options=('Burkina Faso', 'Cameroon', 'Mali', 'Niger…

In [13]:
# Now save the time series of each country keeping as indeces for each time series the own first and last index.
def save(group, name):
    country = group.name
    group = group[country]
    # Adjust time series group.
    first_idx = group.first_valid_index()
    last_idx = group.last_valid_index()
    group = group.loc[first_idx:last_idx]
    # Save.
    group.to_csv(PATH_TO_SAVE_DATA + country + "/" + name + ".csv", index_label = False)

In [14]:
# Save the dataframes.
df.groupby(level = "Country", axis = 1).apply(lambda x: save(x, name = "wfp_ramadan"));

## Adjustment of the Ramadan time series to the time coverage of the FCG <= 2 analysis

Now, we manage the current data keeping only the administrative regions present in the FCG <= 2 analysis. Furthermore, we aggregate data following the FCG <= 2 data collection: the FCG <= 2 value of a certain province corresponding to a referece date does not correspond to the percentage of poor and borderline people on that day, but in a greater time range.

In [15]:
def SvyDate_agg(group):
    country = group.name[0]
    adminstrata = group.name[1]
    indicator = group.name[2]
    group = group.loc[group.first_valid_index():group.last_valid_index()]
    # Load the information regarding the FCG <= 2 survey.
    fcs_coverage = pd.read_csv(f"../1. Food Consumption Score (FCS)/output_timeseries/{country}/wfp_SvyDateRange-interpolate.csv", header = [0, 1], index_col = 0)
    fcs_coverage.index = pd.to_datetime(fcs_coverage.index)
    # Ignore the adminstrata not considered in the FCG <= 2 survey.
    if adminstrata in fcs_coverage.columns.get_level_values("AdminStrata").unique():
        fcs_coverage_adminstrata = fcs_coverage[adminstrata].loc[fcs_coverage[adminstrata].first_valid_index():fcs_coverage[adminstrata].last_valid_index()]
        # Sliding window according to the coverage of the FCG <= 2 analysis.
        start_svy = fcs_coverage_adminstrata.index[0]
        end_svy = fcs_coverage_adminstrata.index[-1]
        new_group = group.loc[start_svy:end_svy].copy()
        for index in new_group.index:
            look_back = int(fcs_coverage_adminstrata.loc[index].values[0])
            # Window over the column.
            new_group.loc[index] = group.loc[index-(look_back-1)*group.index.freq:index].sum() 
        return new_group
    else:
        pass
    
df_SvyWindow = df.groupby(level = ["Country", "AdminStrata", "Indicator"], axis = 1, group_keys = False).apply(SvyDate_agg)
df_SvyWindow.head()

Country          Burkina Faso                                          \
AdminStrata Boucle-Du-Mouhoun Cascades  Centre Centre-Est Centre-Nord   
Indicator             Ramadan  Ramadan Ramadan    Ramadan     Ramadan   
Datetime                                                                
2018-07-02                NaN      NaN     NaN        NaN         NaN   
2018-07-03                NaN      NaN     NaN        NaN         NaN   
2018-07-04                NaN      NaN     NaN        NaN         NaN   
2018-07-05                NaN      NaN     NaN        NaN         NaN   
2018-07-06                NaN      NaN     NaN        NaN         NaN   

Country                                                            ...  \
AdminStrata Centre-Ouest Centre-Sud     Est Hauts-Bassins    Nord  ...   
Indicator        Ramadan    Ramadan Ramadan       Ramadan Ramadan  ...   
Datetime                                                           ...   
2018-07-02           NaN        NaN     NaN           NaN     NaN  ...   
2018-07-03           NaN        NaN     NaN           NaN     NaN  ...   
2018-07-04           NaN        NaN     NaN           NaN     NaN  ...   
2018-07-05           NaN        NaN     NaN           NaN     NaN  ...   
2018-07-06           NaN        NaN     NaN           NaN     NaN  ...   

Country       Yemen                                                          \
AdminStrata  Hajjah     Ibb    Lahj   Marib  Raymah  Sa'ada  Sana'a Shabwah   
Indicator   Ramadan Ramadan Ramadan Ramadan Ramadan Ramadan Ramadan Ramadan   
Datetime                                                                      
2018-07-02     13.0    13.0    13.0    13.0    13.0    13.0    13.0    13.0   
2018-07-03     12.0    12.0    12.0    12.0    12.0    12.0    12.0    12.0   
2018-07-04     11.0    11.0    11.0    11.0    11.0    11.0    11.0    11.0   
2018-07-05     10.0    10.0    10.0    10.0    10.0    10.0    10.0    10.0   
2018-07-06      9.0     9.0     9.0     9.0     9.0     9.0     9.0     9.0   

Country                      
AdminStrata Socotra   Taizz  
Indicator   Ramadan Ramadan  
Datetime                     
2018-07-02     13.0    13.0  
2018-07-03     12.0    12.0  
2018-07-04     11.0    11.0  
2018-07-05     10.0    10.0  
2018-07-06      9.0     9.0  

[5 rows x 75 columns]

In [16]:
# Plot static time series.
TsIP(df_SvyWindow).interactive_plot_df(matplotlib = False, style = "lines", yaxis = "Ramadan", first_last_valid_index_group = True)

interactive(children=(ToggleButtons(description='Country', options=('Burkina Faso', 'Cameroon', 'Mali', 'Niger…

In [17]:
# Save the static time series.
df_SvyWindow.groupby(level = "Country", axis = 1).apply(lambda x: save(x, name = "wfp_ramadan_SvyWindow"));